In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
import datetime
warnings.filterwarnings('ignore')
%matplotlib inline

# 自动化email

In [2]:
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.header import Header
from smtplib import SMTP_SSL


class AutEmail(object):
    
    def __init__(self):
        
        self.host_server = 'smtp.exmail.qq.com'
        self.username = 'yanghua@tuia.cn'
        self.password = 'Yh1995'
        
        self.smtp = SMTP_SSL(self.host_server)#create instance smtp
        self.smtp.set_debuglevel(1)
        self.smtp.ehlo(self.host_server)
        self.smtp.login(self.username,self.password)#login
        
    
    def attach_file(self,receiver):
        
        sender = self.username
        
        msg = MIMEMultipart('related')
        msg['Subject'] = Header('华东AE,直客销售周数据','utf-8')
        msg["From"] = sender
        msg["To"] = ",".join([receiver])
        
        html_msg = \
        """
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <title>Title</title>
        </head>
        <body>
            <p1>HI,</p1>
            <p1>这是上周华东AE和直客销售的周数据，注意查收</p1>
        </body>
        </html>
        """
        # html 内容
        content_html = MIMEText(html_msg, "html", "utf-8")
        msg.attach(content_html)
        
        attach_table_one = MIMEApplication(open(r'C:/Users/DELL/Desktop/Task/zhike/aeresult.xlsx', 'rb').read())
        # 给附件增加标题
        attach_table_one.add_header('Content-Disposition', 'attachment',filename='华东AE周数据.xlsx')
        #  这样的话，附件名称就可以是中文的了，不会出现乱码
        attach_table_one.set_charset('utf-8')
        msg.attach(attach_table_one)

        attach_table_two = MIMEApplication(open(r'C:/Users/DELL/Desktop/Task/zhike/saleresult.xlsx', 'rb').read())
        # 给附件增加标题
        attach_table_two.add_header('Content-Disposition', 'attachment',filename='直客销售周数据.xlsx')
        #  这样的话，附件名称就可以是中文的了，不会出现乱码
        attach_table_two.set_charset('utf-8')
        msg.attach(attach_table_two)

        self.smtp.sendmail(sender,receiver,msg.as_string())
        self.smtp.quit()
            
        return 

# 双周会

In [3]:
"""
Data Source = 平台广告全部配置数据；每周新上线广告主
Filter = 互动广告

"""
app_ad_df = pd.read_excel('C:/Users/DELL/Desktop/Task/doubleweek/平台广告全部配置数据.xlsx',sheet_name=0,header=0,
                          names=['dt','ader','adid','packid','packname','apppackname','industry',
                                 'tuoguang','wisebool','xh','fq','jfdj','ctr','zhpv','cpc','cvr',
                                 'arpu','aeneam','aeregion','budget','a_fee','chargetype',
                                 'xh_ratio','adcatg','payadcount'])
new_ader_df = pd.read_excel('C:/Users/DELL/Desktop/Task/doubleweek/每周新上线广告主.xlsx',sheet_name=0,header=0,
                            names=['dt','aderid','ader','adid','adname','industry','xh','ae','aeregion'])

In [4]:
app_ad_df.head()

,dt,ader,adid,packid,packname,apppackname,industry,tuoguang,wisebool,xh,...,cvr,arpu,aeneam,aeregion,budget,a_fee,chargetype,xh_ratio,adcatg,payadcount
0,2019年01月28日,兑吧free(2),5127,0,默认,NaN,视频,人工投放,其他,0.0,...,0.0,0.0,胡迪欣,其他,0.0,0.0,cpc,0.0,互动广告,460
1,2019年01月28日,兑吧free(2),12436,0,默认,NaN,视频,人工投放,其他,0.0,...,0.0,0.0,胡迪欣,其他,0.0,0.0,cpc,0.0,互动广告,460
2,2019年01月28日,兑吧free(2),13396,0,默认,NaN,其他服务,人工投放,其他,0.0,...,0.0,0.0,胡迪欣,其他,0.0,0.0,cpc,0.0,互动广告,460
3,2019年01月28日,兑吧free(2),14082,0,默认,NaN,应用分发,人工投放,其他,0.0,...,0.0,0.0,胡迪欣,其他,0.0,0.0,cpc,0.0,互动广告,460
4,2019年01月28日,兑吧free(2),16475,0,默认,NaN,视频,人工投放,其他,0.0,...,0.0,0.0,胡迪欣,其他,0.0,0.0,cpc,0.0,互动广告,460


In [5]:
class AppAdvertMeeting(object):
    
    def __init__(self,app_ad_df,new_ader_df):
        self.app_ad_df = app_ad_df
        self.new_ader_df = new_ader_df
        self.last_week = (datetime.date.today()-datetime.timedelta(days=7)).strftime("%m.%d")+'-'+(datetime.date.today()-datetime.timedelta(days=1)).strftime("%m.%d")
        self.two_weeks_ago = (datetime.date.today()-datetime.timedelta(days=14)).strftime("%m.%d")+'-'+(datetime.date.today()-datetime.timedelta(days=8)).strftime("%m.%d")
        
    def preprocess_df(self,df):
        
        data = df.copy()
        
        data['new_dt'] = data['dt'].apply(lambda x : re.sub(r'年|月|日','-',str(x))[:-1])
        data['dt_cluster'] = data['new_dt'].apply(lambda x : self.two_weeks_ago if (x>=(datetime.date.today()-datetime.timedelta(days=14)).strftime("%Y-%m-%d"))&(x<=(datetime.date.today()-datetime.timedelta(days=8)).strftime("%Y-%m-%d")) else self.last_week)
        data['new_dt'] = data['new_dt'].astype('datetime64')
        data['weekday'] = data['new_dt'].apply(lambda x : x.weekday()+1)
        
        return data
    
    def total_data(self):
        
        data = self.preprocess_df(self.app_ad_df)
        data = data[(data.xh!=0)&(data['adcatg']=='互动广告')]
        
        data_gb = data.groupby('dt_cluster')['xh','fq'].sum().reset_index()
        data_gb['arpu'] = data_gb['xh']/data_gb['fq']
        
        return data_gb
    
    def top_ader(self,jfbool='yes'):
        
        if jfbool == 'yes':
            data = self.preprocess_df(self.app_ad_df)
            data = data[(data.xh!=0)&(data['adcatg']=='互动广告')]
        else:
            data = self.preprocess_df(self.app_ad_df)
            data = data[(data.xh!=0)&(data['adcatg']=='互动广告')&(data['industry']!='JF')]

        data_xh_pt = data.pivot_table(index='ader',columns='dt_cluster',values='xh',aggfunc='sum').fillna(0).reset_index()
        
        data_xh_pt[self.two_weeks_ago+'_rank'] = data_xh_pt.loc[:,self.two_weeks_ago].rank(ascending=False,method='dense')
        data_xh_pt[self.last_week+'_rank'] = data_xh_pt.loc[:,self.last_week].rank(ascending=False,method='dense')
        
        data_fq_pt = data.pivot_table(index='ader',columns='dt_cluster',values='fq',aggfunc='sum').fillna(0).reset_index()

        data_mg = data_xh_pt.merge(data_fq_pt,how='left',on='ader',suffixes=('_xh','_fq'))
        #arpu
        data_mg[self.two_weeks_ago+'_arpu'] = data_mg[self.two_weeks_ago+'_xh']/data_mg[self.two_weeks_ago+'_fq']
        data_mg[self.last_week+'_arpu'] = data_mg[self.last_week+'_xh']/data_mg[self.last_week+'_fq']
        #avg_xh
        data_mg['avg'+self.two_weeks_ago+'_xh'] = data_mg[self.two_weeks_ago+'_xh']/7
        data_mg['avg'+self.last_week+'_xh'] = data_mg[self.last_week+'_xh']/7
        #NaN to zero
        data_mg = data_mg.fillna(0)
        #rank and arpu change
#         data_mg['rank'] = data_mg[self.two_weeks_ago+'_rank'].round(0).astype('str')+'-----'+data_mg[self.last_week+'_rank'].round(0).astype('str')
        data_mg['arpu'] = data_mg[self.two_weeks_ago+'_arpu'].round(3).astype('str')+'-'+data_mg[self.last_week+'_arpu'].round(3).astype('str')

        top_ader = data_mg[['ader',self.two_weeks_ago+'_rank',self.last_week+'_rank','avg'+self.two_weeks_ago+'_xh','avg'+self.last_week+'_xh','arpu']]

        return top_ader.sort_values(['avg'+self.last_week+'_xh'],ascending=[False])
    
    def new_ader(self):
        
        data1 = self.preprocess_df(self.new_ader_df)
        data2 = self.preprocess_df(self.app_ad_df)
        
        ader_ind_dt = data1[['ader','industry','dt_cluster']].drop_duplicates().reset_index(drop=True)
        
        source_df =  ader_ind_dt.merge(data2.groupby(['dt_cluster','ader','industry'])['xh'].sum().reset_index(),on=['dt_cluster','ader','industry'],how='left')
        
        ader_xh_gb = source_df.groupby(['dt_cluster'])['ader','xh'].agg({'ader':'count','xh':'sum'}).reset_index()
        ader_xh_gb['avg_xh'] = ader_xh_gb['xh']/7
        
        ind_amount_df = source_df.groupby(['dt_cluster','industry'])['ader','xh'].agg({'ader':'count','xh':'sum'}).reset_index()
        
        dict_list = [pd.DataFrame(
            dict(dt_cluster=str(i),ind='、'.join(map(str,ind_amount_df[ind_amount_df.dt_cluster==i].nlargest(5,'xh')['industry'].tolist())),amount='、'.join(map(str,ind_amount_df[ind_amount_df.dt_cluster==i].nlargest(1,'ader')['ader'].tolist())))
            ,index=[0]) for i in pd.unique(ind_amount_df['dt_cluster'])]
        
        ind_amount = pd.concat(dict_list)
        """
        ValueError: If using all scalar values, you must pass an index
        
        Perhaps it is because the order of items in a list in Python are persistent whereas the ordering of items in a dictionary are not. 
        You can instantiate a DataFrame with an empty dictionary. 
        In principle I suppose a single-row DataFrame as shown here would also be ok to build from a dictionary because the order does not matter (but this hasn't been implemented). 
        However with multiple rows, Pandas would not be able to make a DataFrame because it would not know which items belonged to the same row
        
        url:https://stackoverflow.com/questions/17839973/constructing-pandas-dataframe-from-values-in-variables-gives-valueerror-if-usi
        """

        return ader_xh_gb,ind_amount,source_df[source_df.dt_cluster==self.last_week].sort_values('xh',ascending=False).head(15)
    
    def result(self):
        
        data_gb = self.total_data()
        top_ader = self.top_ader()
        ader_xh_gb,ind_amount,_ = self.new_ader()
        
        top_ten_ader = pd.DataFrame(
            dict(dt_cluster=[self.last_week,self.two_weeks_ago],
                 top_ten_ader_xh=[top_ader.nlargest(10,'avg'+self.last_week+'_xh')['avg'+self.last_week+'_xh'].sum(),top_ader.nlargest(10,'avg'+self.two_weeks_ago+'_xh')['avg'+self.two_weeks_ago+'_xh'].sum()])
        )
        total_mg = data_gb.merge(top_ten_ader,on='dt_cluster',how='left').merge(ader_xh_gb[['dt_cluster','avg_xh','xh']],on='dt_cluster',how='left',suffixes=('','_newader'))
        total_mg['xh_newader_ratio'] = total_mg['xh_newader']/total_mg['xh']
        total_mg.drop(labels=['fq','xh_newader'],inplace=True,axis=1)
        
        new_ader_mg = ader_xh_gb.merge(ind_amount,on='dt_cluster',how='left')
        new_ader_mg.drop(labels=['avg_xh'],inplace=True,axis=1)
        
        return total_mg,new_ader_mg
    
    def week_contrast(self):
        
        data = self.preprocess_df(self.app_ad_df)        
        data_pt = data.pivot_table(index='weekday',columns='dt_cluster',values='xh',aggfunc='sum').reset_index()
        
        return data_pt
        

# 周数据

In [15]:
weekdf = pd.read_excel('C:/Users/DELL/Desktop/Task/week/广告大盘数据子母表.xlsx',header=0,
                       names=['dt','type','catg','industry','ader','resource','fq','bg','jfdj','xh'])

In [16]:
weekdf.head()

,dt,type,catg,industry,ader,resource,fq,bg,jfdj,xh
0,2019年02月15日,-1,0,JF,兑吧free(2),--,3357,3308,264,0.0
1,2019年02月15日,-1,0,JF,兑吧free(3),--,5084,4989,330,0.0
2,2019年02月15日,-1,0,会籍卡,兑吧free(2),兑吧,11187,11486,1351,0.0
3,2019年02月15日,-1,0,会籍卡,谛听网络技术（杭州）有限公司(DK),环球黑卡,0,0,0,0.0
4,2019年02月15日,-1,0,保险,北京聚齐众兴网络科技有限公司(JQ),黑牛保险,0,0,0,0.0


In [17]:
class WeekData(object):
    
    def __init__(self,df):
        self.df = df
        
    def preprocess_df(self,df):
        
        data = df.copy()
        
        data['new_dt'] = data['dt'].apply(lambda x : re.sub(r'年|月|日','-',str(x))[:-1])
        data['new_dt'] = data['new_dt'].astype('datetime64')
        data['weekday'] = data['new_dt'].apply(lambda x : x.weekday()+1)
        data['dt_catg'] = data['new_dt'].apply(lambda x : 'two_week_ago' if x <= pd.unique(data.new_dt)[6] else 'last_week')#distinguish two week ago and last week
                                             
        return data
    
    def industry(self):
        
        data = self.preprocess_df(self.df)
        
        """
        three section include xh,jfdj,bg
        
        """
        
        data_xh_section = data.pivot_table(index='industry',columns='dt_catg',values='xh',aggfunc='sum',margins=True).reset_index()
        data_xh_section['lw_avg_xh'] = (data_xh_section['last_week']/7).round(0)
        data_xh_section['lw_xh_ratio'] = (data_xh_section['last_week'][0:-1]/int(data_xh_section['last_week'][-1:])).round(4)
        
        data_bg_section = data[data.dt_catg=='last_week'].pivot_table(index='industry',columns='catg',values='bg',aggfunc='sum',margins=True).reset_index()
        data_bg_section['pay_ratio'] = (data_bg_section[1]/data_bg_section['All']).round(4)  
        
        data_jfdj_section = data[data.dt_catg=='last_week'].groupby('industry')['jfdj'].sum().reset_index()
        
        data_ader_section = (data[(data.xh!=0)][['dt_catg','industry','ader']].drop_duplicates()).pivot_table(index='industry',columns='dt_catg',values='ader',aggfunc='count').reset_index()

        """
        merge three section
        
        """
        
        data_mg = ((data_xh_section[['industry','last_week','two_week_ago','lw_avg_xh','lw_xh_ratio']].merge(data_bg_section[['industry',1,'pay_ratio']],on='industry')).merge(data_jfdj_section,on='industry')).merge(data_ader_section,on='industry',suffixes=('_xh','_count'))
        
        """
        process merged data
        
        """
        data_mg['pay_arpu'] = data_mg['last_week_xh']/data_mg[1]
        data_mg['cpc'] = data_mg['last_week_xh']/data_mg['jfdj']
        result = data_mg.sort_values('lw_avg_xh',ascending=False)
    
        return result[result.lw_avg_xh!=0]
    
    def top_ten_ader(self):
        
        data = self.preprocess_df(self.df)
        
        data_xh_section = data.pivot_table(index='resource',columns='dt_catg',values='xh',aggfunc='sum',margins=True).reset_index()
        data_xh_section['lw_avg_xh'] = (data_xh_section['last_week']/7).round(0)
        
        data_bg_section = data[data.dt_catg=='last_week'].pivot_table(index='resource',columns='catg',values='bg',aggfunc='sum',margins=True).reset_index()
        
        data_jfdj_section = data[data.dt_catg=='last_week'].groupby('resource')['jfdj'].sum().reset_index()
        
        data_mg = (data_xh_section[['resource','last_week','two_week_ago','lw_avg_xh']].merge(data_bg_section[['resource',1]],on='resource')).merge(data_jfdj_section,on='resource')

        data_mg['pay_arpu'] = data_mg['last_week']/data_mg[1]
        data_mg['cpc'] = data_mg['last_week']/data_mg['jfdj']
        
        result = data_mg[['resource','lw_avg_xh',1,'last_week','two_week_ago','cpc','pay_arpu']].nlargest(10,'lw_avg_xh')
        
        return result
    
    def analysis(self):
        
        data = self.preprocess_df(self.df)
        
        ind_al = data.pivot_table(index='industry',columns='dt_catg',values='xh',aggfunc='sum').reset_index()
        ind_al['xhdiff'] = ind_al['last_week']/7 - ind_al['two_week_ago']/7
        ind_res = data.pivot_table(index=['industry','resource'],columns='dt_catg',values='xh',aggfunc='sum').reset_index()
        ind_res['xhdiff'] = ind_res['last_week']/7 - ind_res['two_week_ago']/7
        
        res_ader = data.pivot_table(index=['industry','resource','ader'],columns='dt_catg',values='xh',aggfunc='sum').reset_index()
        res_ader['xhdiff'] = res_ader['last_week']/7 - res_ader['two_week_ago']/7
        
        for i in ind_al[(ind_al.xhdiff>50000)]['industry'].tolist():
            a = ind_res[ind_res.industry==i]
            b = res_ader[res_ader.industry==i].nlargest(10,'xhdiff')
            print(a)
            print(b)
        
        print('--------------')
            
        for j in ind_al[(ind_al.xhdiff<-50000)]['industry'].tolist():
            c = ind_res[ind_res.industry==j]
            d = res_ader[res_ader.industry==j].nsmallest(10,'xhdiff')
            print(c)
            print(d)
        
        return 

In [18]:
class_WeekData = WeekData(weekdf)
class_WeekData.analysis()

dt_catg industry resource   last_week  two_week_ago        xhdiff
119         电商平台       --        0.00          0.00      0.000000
120         电商平台       兑吧        0.00          0.00      0.000000
121         电商平台     内部项目        0.00          0.00      0.000000
122         电商平台     天天趣闻     5105.59       3111.07    284.931429
123         电商平台      奥买家   184022.55      14926.88  24156.524286
124         电商平台       待定     1369.50           NaN           NaN
125         电商平台      拼多多  7480824.86    6849862.78  90137.440000
126         电商平台     本来生活        0.00          0.00      0.000000
127         电商平台       淘宝        0.00          0.00      0.000000
128         电商平台    电商购物券      289.80           NaN           NaN
129         电商平台      趣专享     2398.54       3705.50   -186.708571
130         电商平台      返利网        0.00          0.00      0.000000
dt_catg industry resource                       ader   last_week  \
402         电商平台      拼多多      杭州微米网络科技有限公司(Android)  7480824.86   
400   

In [19]:
class_WeekData.industry()

,industry,last_week_xh,two_week_ago_xh,lw_avg_xh,lw_xh_ratio,1,pay_ratio,jfdj,last_week_count,two_week_ago_count,pay_arpu,cpc
0,JF,9462227.03,13612621.72,1351747.0,0.3033,58506247.0,0.9992,20833687,105.0,109.0,0.161730,0.454179
15,电商平台,7674010.84,6871606.23,1096287.0,0.2459,62780483.0,0.8424,28216443,7.0,4.0,0.122236,0.271969
18,货到付款,2903676.73,2229742.02,414811.0,0.0931,28935410.0,0.9901,9715512,44.0,31.0,0.100350,0.298870
19,贷款,2885792.72,2863079.25,412256.0,0.0925,38937761.0,0.9261,4761472,20.0,16.0,0.074113,0.606072
4,信用卡,2235628.43,2503956.18,319375.0,0.0716,20126589.0,0.9744,3586791,47.0,43.0,0.111078,0.623295
3,保险,1600057.43,1438244.57,228580.0,0.0513,15404547.0,0.8619,5323232,15.0,13.0,0.103869,0.300580
12,游戏,1176166.84,1364669.30,168024.0,0.0377,20011944.0,0.9924,3842115,9.0,9.0,0.058773,0.306125
8,应用分发,1070895.14,1091297.21,152985.0,0.0343,57539801.0,0.9655,10229895,15.0,16.0,0.018611,0.104683
20,运营商,979481.22,1234145.46,139926.0,0.0314,5920613.0,0.9817,2167349,13.0,14.0,0.165436,0.451926
2,会籍卡,401276.56,458773.25,57325.0,0.0129,2164473.0,0.9715,576326,13.0,11.0,0.185392,0.696267


In [20]:
class_WeekData.top_ten_ader()

,resource,lw_avg_xh,1,last_week,two_week_ago,cpc,pay_arpu
6,jf,1351685.0,58501733.0,9461797.43,13611238.70,0.454249,0.161735
53,拼多多,1068689.0,57813983.0,7480824.86,6849862.78,0.304309,0.129395
87,货到付款,413656.0,28877403.0,2895593.43,2229741.42,0.300720,0.100272
3,360借条,344543.0,15278488.0,2411799.75,2385887.72,0.647946,0.157856
52,招商银行信用卡,187128.0,11561237.0,1309897.50,1663437.23,0.592001,0.113301
51,战神破晓,145099.0,13511400.0,1015694.63,1266803.96,0.483156,0.075173
12,中国移动流量,135371.0,5430043.0,947594.23,1155540.54,0.465541,0.174510
56,星球联盟,73824.0,7397530.0,516766.29,699323.02,0.183282,0.069857
54,支付宝,64931.0,15544557.0,454515.26,306190.88,0.067825,0.029240
65,浦发银行信用卡,61606.0,3197477.0,431242.07,267922.48,0.834384,0.134869
